In [1]:
import keras
from keras.applications import VGG19, InceptionV3
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np

from keras.layers.convolutional import Conv2D
from keras.engine.topology import InputLayer
from keras.layers.pooling import MaxPooling2D

from attr import attrs, attrib

Using TensorFlow backend.


In [15]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [2]:
im_size = (224, 224, 3)

random_img = np.random.uniform(size=im_size)
model_input = keras.layers.Input(shape=im_size)

In [3]:
trained_vgg = VGG19(input_tensor=model_input)
untrained_vgg = VGG19(input_tensor=model_input, weights=None)

In [4]:
@attrs
class GraphableNeruon(object):
    weights = attrib()
    bias = attrib()
    layer = attrib()
    neuron_number = attrib()
    layer_type = attrib()




In [5]:
def get_neurons(net):
    neurons = []
    for i, layer in enumerate(net.layers):
        if type(layer) is Conv2D:
            wts, bias = layer.get_weights()
            for nn, (w, b) in enumerate(zip(wts.T, bias)):
                neurons.append(GraphableNeruon(w, b, i, nn, layer.get_config()))
    return neurons

In [18]:
def set_neurons(neurons, net):
    last_neuron = neurons[0]
    layer = net.layers[last_neuron.layer]
    layer_weights, layer_bias = layer.get_weights()
    layer_weights = layer_weights.T
    for n in neurons:
        if not last_neuron.layer == n.layer:
            layer.set_weights([layer_weights.T, layer_bias])
            layer = net.layers[n.layer]
            layer_weights, layer_bias = layer.get_weights()
            layer_weights = layer_weights.T
            last_neuron = n
        
        layer_weights[n.neuron_number] = n.weights
        layer_bias[n.neuron_number] = n.bias
        
    
        

In [19]:
%lprun -f set_neurons set_neurons(get_neurons(trained_vgg), trained_vgg)

In [8]:
6*[[]]

[[], [], [], [], [], []]

In [9]:
a = trained_vgg.layers[1]

In [10]:
trained_vgg.layers[1].set_weights

<bound method Layer.set_weights of <keras.layers.convolutional.Conv2D object at 0x7f0978059dd8>>

In [11]:
trained_vgg.layers[0].set_weights

<bound method Layer.set_weights of <keras.engine.topology.InputLayer object at 0x7f089c0b50b8>>

In [12]:
a.set_weights?

In [13]:
%load_ext line_profiler

In [14]:
!conda install line_profiler

Fetching package metadata .......
Solving package specifications: ..........

Package plan for installation in environment /home/dhash/anaconda3:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    anaconda-custom            |           py35_0           3 KB
    requests-2.13.0            |           py35_0         811 KB
    line_profiler-2.0          |           py35_0         131 KB
    pyopenssl-16.2.0           |           py35_0          70 KB
    conda-4.3.16               |           py35_0         510 KB
    ------------------------------------------------------------
                                           Total:         1.5 MB

The following NEW packages will be INSTALLED:

    line_profiler: 2.0-py35_0       

The following packages will be UPDATED:

    anaconda:      4.2.0-np111py35_0 --> custom-py35_0
    conda:         4.2.13-py35_0     --> 4.3.16-py35_0
    pyopenssl:   